<a href="https://colab.research.google.com/github/madhusudhanrao-ppm/devrel-colab/blob/main/RAGADK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install "oci[adk]" oci langchain-oracledb langchain

In [ ]:
import os
oci_dir = '/root/.oci'
if not os.path.exists(oci_dir):
    os.makedirs(oci_dir)

from google.colab import drive
drive.mount('/content/drive')

In [5]:
# Copy OCI config file to Colab environment
!cp /content/drive/MyDrive/OCI_KEY/config ~/.oci/config
# Copy private key file to Colab environment
!cp /content/drive/MyDrive/OCI_KEY/oradbclouducm_sso.pem $oci_dir/oradbclouducm_sso.pem

In [11]:
import json
import os
from typing import Dict, List, Optional, Any
from datetime import datetime
from oci.addons.adk import Agent, AgentClient, tool
from oci.object_storage import ObjectStorageClient
import oci
import asyncio
import nest_asyncio

In [14]:
class DocumentStore:
    """Simple in-memory document store for RAG"""

    def __init__(self):
        self.documents = {
            "oracle_cloud_overview": {
                "title": "Oracle Cloud Infrastructure Overview",
                "content": """Oracle Cloud Infrastructure (OCI) is a secure, economical and high-performing
                cloud platform. OCI provides a complete suite of cloud computing services including compute,
                storage, networking, database, analytics, and AI services."""
            },
            "adk_intro": {
                "title": "Oracle Agent Development Toolkit",
                "content": """The Oracle Agent Development Toolkit (ADK) enables developers to create
                intelligent agents that can use tools, reason about tasks, and interact with various services.
                Agents can be deployed as endpoints and invoked via API."""
            },
            "rag_basics": {
                "title": "RAG (Retrieval-Augmented Generation) Basics",
                "content": """RAG combines document retrieval with generative AI. When a user asks a question,
                the system retrieves relevant documents from a knowledge base and uses them to generate
                accurate, contextual responses."""
            },
            "oci_storage": {
                "title": "OCI Object Storage",
                "content": """OCI Object Storage is a fully managed cloud storage service for storing
                unstructured data. It provides secure, durable, and scalable storage with multiple
                storage tiers and lifecycle policies."""
            },
            "oci_database": {
                "title": "OCI Database Services",
                "content": """OCI offers multiple database services including Autonomous Database,
                MySQL Database, PostgreSQL Database, and NoSQL Database. Each service is optimized
                for different use cases and provides high availability."""
            }
        }

    def search_documents(self, query: str) -> List[Dict[str, str]]:
        """Simple keyword-based document search"""
        query_lower = query.lower()
        results = []

        for doc_id, doc in self.documents.items():
            title_match = query_lower in doc["title"].lower()
            content_match = query_lower in doc["content"].lower()

            if title_match or content_match:
                results.append({
                    "id": doc_id,
                    "title": doc["title"],
                    "preview": doc["content"][:200] + "..."
                })

        return results[:3]  # Return top 3 results


# Global document store instance
doc_store = DocumentStore()


# ============================================================================
# TOOL DEFINITIONS FOR THE AGENT
# ============================================================================

@tool
def retrieve_documents(query: str) -> Dict[str, Any]:
    """
    Retrieve relevant documents from the knowledge base based on the query.

    Args:
        query(str): The search query to find relevant documents

    Returns:
        dict: Retrieved documents with metadata
    """
    results = doc_store.search_documents(query)

    if not results:
        return {
            "status": "not_found",
            "message": f"No documents found for query: '{query}'",
            "documents": []
        }

    return {
        "status": "success",
        "message": f"Found {len(results)} relevant document(s)",
        "documents": results,
        "timestamp": datetime.now().isoformat()
    }


@tool
def get_document_content(document_id: str) -> Dict[str, Any]:
    """
    Get the full content of a document by its ID.

    Args:
        document_id(str): The ID of the document to retrieve

    Returns:
        dict: Full document content or error message
    """
    if document_id not in doc_store.documents:
        return {
            "status": "error",
            "message": f"Document not found: {document_id}"
        }

    doc = doc_store.documents[document_id]
    return {
        "status": "success",
        "id": document_id,
        "title": doc["title"],
        "content": doc["content"],
        "retrieved_at": datetime.now().isoformat()
    }


@tool
def search_oci_knowledge_base(topic: str, category: str = "all") -> Dict[str, Any]:
    """
    Search the OCI knowledge base for information.

    Args:
        topic(str): The topic to search for
        category(str): Category filter (all, compute, storage, database, ai)

    Returns:
        dict: Search results with relevant information
    """
    knowledge_base = {
        "compute": [
            "VM instances", "Container instances", "Functions",
            "Kubernetes Engine", "Bare Metal Compute"
        ],
        "storage": [
            "Object Storage", "Block Volumes", "File Storage",
            "Archive Storage", "Data Transfer"
        ],
        "database": [
            "Autonomous Database", "MySQL Database", "PostgreSQL Database",
            "NoSQL Database", "Database Backup"
        ],
        "ai": [
            "AI Services", "Generative AI", "Document Understanding",
            "Anomaly Detection", "Data Science"
        ]
    }

    results = []
    topic_lower = topic.lower()

    if category == "all":
        categories = knowledge_base.keys()
    else:
        categories = [category] if category in knowledge_base else []

    for cat in categories:
        for item in knowledge_base[cat]:
            if topic_lower in item.lower():
                results.append({"category": cat, "item": item})

    return {
        "topic": topic,
        "category": category,
        "results": results,
        "count": len(results)
    }


@tool
def get_conversation_context(session_id: str) -> Dict[str, Any]:
    """
    Get conversation context information for maintaining state.

    Args:
        session_id(str): The session identifier

    Returns:
        dict: Conversation context and history metadata
    """
    return {
        "session_id": session_id,
        "start_time": datetime.now().isoformat(),
        "context": {
            "user_intent": "general_inquiry",
            "topics_discussed": [],
            "documents_referenced": []
        },
        "status": "active"
    }


# ============================================================================
# RAG CHATBOT CLASS
# ============================================================================

class RAGChatbot:
    """
    RAG-enabled conversational AI chatbot using Oracle ADK
    """

    def __init__(
        self,
        agent_endpoint_id: str,
        auth_type: str = "api_key",
        profile: str = "DEFAULT",
        region: str = "us-chicago-1"
    ):
        """
        Initialize the RAG chatbot.

        Args:
            agent_endpoint_id (str): The OCID of the agent endpoint
            auth_type (str): Authentication type (api_key, instance_principal, resource_principal)
            profile (str): OCI config profile name
            region (str): OCI region
        """
        self.agent_endpoint_id = agent_endpoint_id
        self.auth_type = auth_type
        self.profile = profile
        self.region = region
        self.conversation_history = []
        self.agent = None
        self.client = None

        # Initialize the agent client
        self._initialize_client()

    def _initialize_client(self):
        """Initialize the OCI Agent Client"""
        try:
            self.client = AgentClient(
                auth_type=self.auth_type,
                profile=self.profile,
                region=self.region
            )
            print(f"✓ Agent client initialized")
            print(f"  - Region: {self.region}")
            print(f"  - Auth Type: {self.auth_type}")
        except Exception as e:
            print(f"✗ Failed to initialize agent client: {e}")
            raise

    def setup_agent(self, instructions: str = None):
        """
        Set up the agent with instructions and tools.

        Args:
            instructions (str): Custom instructions for the agent
        """
        if instructions is None:
            instructions = """You are a helpful RAG-powered assistant for Oracle Cloud Infrastructure.
            Your role is to:
            1. Answer questions about OCI services and features
            2. Retrieve relevant documents from the knowledge base when needed
            3. Provide accurate, context-aware responses based on retrieved information
            4. Guide users through OCI concepts and best practices

            When answering questions:
            - Always search for relevant documents first
            - Cite the documents you used
            - Provide clear, structured answers
            - Ask clarifying questions if needed"""

        try:
            self.agent = Agent(
                client=self.client,
                agent_endpoint_id=self.agent_endpoint_id,
                instructions=instructions,
                tools=[
                    retrieve_documents,
                    get_document_content,
                    search_oci_knowledge_base,
                    get_conversation_context
                ]
            )

            # Sync instructions and tools to remote agent
            self.agent.setup()
            print("✓ Agent setup complete with RAG tools")

        except Exception as e:
            print(f"✗ Failed to setup agent: {e}")
            raise

    def chat(self, user_input: str) -> Dict[str, Any]:
        """
        Process a user message and generate a response using the agent.

        Args:
            user_input (str): The user's message

        Returns:
            dict: Response with agent output and metadata
        """
        if not self.agent:
            return {
                "status": "error",
                "message": "Agent not initialized. Call setup_agent() first."
            }

        try:
            # Add to conversation history
            self.conversation_history.append({
                "role": "user",
                "content": user_input,
                "timestamp": datetime.now().isoformat()
            })

            print(f"\n📝 User: {user_input}")
            print("-" * 60)

            # Run the agent
            response = self.agent.run(user_input)

            # Store response in history
            response_text = str(response)
            self.conversation_history.append({
                "role": "assistant",
                "content": response_text,
                "timestamp": datetime.now().isoformat()
            })

            return {
                "status": "success",
                "user_input": user_input,
                "response": response,
                "turn": len(self.conversation_history) // 2
            }

        except Exception as e:
            print(f"✗ Error processing message: {e}")
            return {
                "status": "error",
                "message": str(e),
                "user_input": user_input
            }

    def get_conversation_history(self) -> List[Dict[str, Any]]:
        """Get the conversation history"""
        return self.conversation_history

    def clear_history(self):
        """Clear the conversation history"""
        self.conversation_history = []
        print("✓ Conversation history cleared")


# ============================================================================
# EXAMPLE USAGE
# ============================================================================

def main():
    """
    Demonstrate the RAG chatbot functionality
    """
    print("=" * 70)
    print("Oracle ADK RAG Chatbot Demo")
    print("=" * 70)

    # Configuration
    AGENT_ENDPOINT_ID = "ocid1.genaiagentendpoint.oc1.us-chicago-1.amaaaaaaknuwtjiadfdoluzq45s4owu7wvvwgtdtwvsau7h4ef75edqbmu5a"
    #REGION = "us-chicago-1"
    REGION = "us-chicago-1"
    AUTH_TYPE = "api_key"
    PROFILE = "DEFAULT_CHICAGO"

    # Initialize the chatbot
    print("\n1. Initializing RAG Chatbot...")
    print("-" * 70)

    try:
        chatbot = RAGChatbot(
            agent_endpoint_id=AGENT_ENDPOINT_ID,
            auth_type=AUTH_TYPE,
            profile=PROFILE,
            region=REGION
        )

        # Setup the agent with tools
        print("\n2. Setting up Agent with RAG Tools...")
        print("-" * 70)
        chatbot.setup_agent()

        # Example conversations
        print("\n3. Conducting Conversations...")
        print("-" * 70)

        sample_queries = [
            "What is Database Sharding?",
            "How does RAG work?",
            "Tell me about OCI Database services",
            "What storage options are available in OCI?"
        ]

        for query in sample_queries:
            response = chatbot.chat(query)

            if response["status"] == "success":
                print(f"\n🤖 Agent Response:")
                response["response"].pretty_print()
            else:
                print(f"\n❌ Error: {response['message']}")

        # Print conversation history
        print("\n4. Conversation History Summary")
        print("-" * 70)
        history = chatbot.get_conversation_history()
        print(f"Total exchanges: {len(history) // 2}")
        for i, msg in enumerate(history):
            role_icon = "👤" if msg["role"] == "user" else "🤖"
            print(f"{role_icon} {msg['role'].upper()}: {msg['content'][:60]}...")

    except Exception as e:
        print(f"\n❌ Failed to run chatbot: {e}")
        print("\nTroubleshooting:")
        print("1. Ensure your OCI config is set up: ~/.oci/config")
        print("2. Update AGENT_ENDPOINT_ID with your actual endpoint OCID")
        print("3. Verify your OCI region is correct")
        print("4. Check your authentication credentials")


def interactive_chat():
    """
    Run an interactive chatbot session
    """
    print("=" * 70)
    print("Interactive RAG Chatbot")
    print("=" * 70)
    print("Type 'exit' to quit, 'history' to see conversation history")
    print()

    AGENT_ENDPOINT_ID = 'ocid1.genaiagentendpoint.oc1.us-chicago-1.amaaaaaaknuwtjiadfdoluzq45s4owu7wvvwgtdtwvsau7h4ef75edqbmu5a'

    try:
        chatbot = RAGChatbot(
            agent_endpoint_id=AGENT_ENDPOINT_ID,
            auth_type="api_key",
            profile="DEFAULT_CHICAGO",
            region="us-chicago-1"
        )

        chatbot.setup_agent()

        while True:
            user_input = input("\n👤 You: ").strip()

            if not user_input:
                continue

            if user_input.lower() == "exit":
                print("\n👋 Goodbye!")
                break

            if user_input.lower() == "history":
                history = chatbot.get_conversation_history()
                print("\n📋 Conversation History:")
                for msg in history:
                    print(f"  {msg['role']}: {msg['content'][:50]}...")
                continue

            response = chatbot.chat(user_input)
            if response["status"] == "success":
                print(f"\n🤖 Assistant: {response['response']}")
            else:
                print(f"\n❌ Error: {response['message']}")

    except Exception as e:
        print(f"\n❌ Chatbot error: {e}")


if __name__ == "__main__":
    # Run the demo
    main()

    # Uncomment to run interactive chat
    # interactive_chat()

Oracle ADK RAG Chatbot Demo

1. Initializing RAG Chatbot...
----------------------------------------------------------------------
✓ Agent client initialized
  - Region: us-chicago-1
  - Auth Type: api_key

2. Setting up Agent with RAG Tools...
----------------------------------------------------------------------


[01/27/26 13:12:32]  INFO     Checking integrity of agent details...

[01/27/26 13:12:33]  INFO     Checking synchronization of local and remote agent settings...

                     INFO     Checking synchronization of local and remote function tools...

╭───────────────────────────────── Local and remote function tools found ─────────────────────────────────╮
│ Local function tools (4):                                                                               │
│ ['get_conversation_context', 'get_document_content', 'retrieve_documents', 'search_oci_knowledge_base'] │
│                                                                                                         │
│ Remote function tools (4):                                                                              │
│ ['get_conversation_context', 'get_document_content', 'retrieve_documents', 'search_oci_knowledge_base'] │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭────────────────────────────────────── Chat request to remote agent: None ───────────────────────────────────────╮
│ (Local --> Remote)                                                                                              │
│                                                                                                                 │
│ user message:                                                                                                   │
│ null                                                                                                            │
│                                                                                                                 │
│ performed actions by client:                                                                                    │
│ [{'action_id': 'c844fabc-2aed-4aad-b304-bbaeb9dcd34e', 'performed_action_type':                                 │
│ 'FUNCTION_CALLING_PERFORMED_ACTION', 'function_call_output': '{"topic": "storage options", "category":          │
│ "storage", "results": [], "count": 0}'}]                                                                        │
│                                                                                                                 │
│ session id:                                                                                                     │
│ ocid1.genaiagentsession.oc1.us-chicago-1.amaaaaaa7mjirbaaviyp6vuj7sh6ph2kmge5hr26ruvic4ucekvhodmeasgq           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────── Chat response from remote agent ────────────────────────────────────────╮
│ (Local <-- Remote)                                                                                              │
│                                                                                                                 │
│ agent message:                                                                                                  │
│ {                                                                                                               │
│     "role": "AGENT",                                                                                            │
│     "content": {                                                                                                │
│         "text": "Oracle Cloud Infrastructure (OCI) offers a range of storage options, including Object Storage, │
│ Block Storage, File Storage, Archive Storage, and Data Transfer Service. These services cater to various        │
│ business needs, such as storing and serving large amounts of unstructured data, providing low-latency,          │
│ high-performance block storage for databases and applications, storing and sharing files, archiving data, and   │
│ transferring large amounts of data to the cloud. The choice of storage option depends on the specific use case  │
│ and requirements of the application or workload.",                                                              │
│         "citations": null,                                                                                      │
│         "paragraph_citations": null                                                                             │
│     },                                                                                                          │
│     "time_created": "2026-01-27T13:14:59.433000+00:00"                                                          │
│ }                                                                                                               │
│                                                                                                                 │
│ required actions for client to take:                                                                            │
│ null                                                                                                            │
│                                                                                                                 │
│ guardrail result:                                                                                               │
│ None                                                                                                            │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


🤖 Agent Response:


╭────────────────────────────────────────────── Agent run response ───────────────────────────────────────────────╮
│ agent text message:                                                                                             │
│ Oracle Cloud Infrastructure (OCI) offers a range of storage options, including Object Storage, Block Storage,   │
│ File Storage, Archive Storage, and Data Transfer Service. These services cater to various business needs, such  │
│ as storing and serving large amounts of unstructured data, providing low-latency, high-performance block        │
│ storage for databases and applications, storing and sharing files, archiving data, and transferring large       │
│ amounts of data to the cloud. The choice of storage option depends on the specific use case and requirements of │
│ the application or workload.                                                                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


4. Conversation History Summary
----------------------------------------------------------------------
Total exchanges: 4
👤 USER: What is Database Sharding?...
🤖 ASSISTANT: raw_responses=[RawResponse(raw_data={'message': None, 'trace...
👤 USER: How does RAG work?...
🤖 ASSISTANT: raw_responses=[RawResponse(raw_data={'message': None, 'trace...
👤 USER: Tell me about OCI Database services...
🤖 ASSISTANT: raw_responses=[RawResponse(raw_data={'message': None, 'trace...
👤 USER: What storage options are available in OCI?...
🤖 ASSISTANT: raw_responses=[RawResponse(raw_data={'message': None, 'trace...
